# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,queenstown,-31.8976,26.8753,11.72,84,40,2.53,ZA,1725488424
1,1,edinburgh of the seven seas,-37.0676,-12.3116,10.34,64,83,6.39,SH,1725489241
2,2,puerto natales,-51.7236,-72.4875,9.25,43,20,17.49,CL,1725489158
3,3,waitangi,-43.9535,-176.5597,9.90,69,6,3.13,NZ,1725489154
4,4,bethel,41.3712,-73.4140,22.46,66,0,1.78,US,1725489183


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    frame_width = 800,
    frame_height = 600
)
# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_temp_min = 20
ideal_temp_max = 30
ideal_humidity_min = 25
ideal_humidity_max = 55
ideal_cloudiness_max = 20
ideal_wind_speed_max = 5

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
ideal_city_df = city_data_df[
    (city_data_df["Max Temp"]>= ideal_temp_min)&
    (city_data_df["Max Temp"]<= ideal_temp_max)&
    (city_data_df["Humidity"]>= ideal_humidity_min)&
    (city_data_df["Humidity"]<= ideal_humidity_max)&
    (city_data_df["Cloudiness"]<= ideal_cloudiness_max)&
    (city_data_df["Wind Speed"]<= ideal_wind_speed_max)
]

ideal_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,richards bay,-28.7830,32.0377,21.00,52,0,4.32,ZA,1725488707
17,17,kalemie,-5.9475,29.1947,23.33,53,15,1.33,CD,1725489259
96,96,ludington,43.9553,-86.4526,24.01,49,0,3.09,US,1725489056
169,169,nchelenge,-9.3451,28.7340,20.55,39,4,3.17,ZM,1725489432
178,178,peace river,56.2501,-117.2860,20.17,39,0,3.60,CA,1725489442
190,190,jamestown,42.0970,-79.2353,23.80,40,0,2.57,US,1725489455
299,299,surab,28.4914,66.2585,22.12,33,0,0.74,PK,1725489580
304,304,nantucket,41.2835,-70.0995,20.06,40,0,2.06,US,1725489586
331,331,uchiza,-8.4592,-76.4633,28.17,47,0,1.14,PE,1725489616
375,375,kapoeta,4.7717,33.5903,25.40,46,6,1.18,SS,1725489666


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
11,richards bay,ZA,-28.7830,32.0377,52,
17,kalemie,CD,-5.9475,29.1947,53,
96,ludington,US,43.9553,-86.4526,49,
169,nchelenge,ZM,-9.3451,28.7340,39,
178,peace river,CA,56.2501,-117.2860,39,
190,jamestown,US,42.0970,-79.2353,40,
299,surab,PK,28.4914,66.2585,33,
304,nantucket,US,41.2835,-70.0995,40,
331,uchiza,PE,-8.4592,-76.4633,47,
375,kapoeta,SS,4.7717,33.5903,46,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",  
    "apiKey": geoapify_key  
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
richards bay - nearest hotel: SUN1 Richards Bay
kalemie - nearest hotel: Hotel du Lac
ludington - nearest hotel: Stearns Hotel
nchelenge - nearest hotel: No hotel found
peace river - nearest hotel: New Ridge Inn
jamestown - nearest hotel: DoubleTree Jamestown
surab - nearest hotel: No hotel found
nantucket - nearest hotel: Faraway Nantucket
uchiza - nearest hotel: No hotel found
kapoeta - nearest hotel: Kuleu
dawson creek - nearest hotel: Comfort Inn
al bawiti - nearest hotel: Old Oasis Hotel
alice springs - nearest hotel: Aurora Alice Springs
balad - nearest hotel: No hotel found
argostolion - nearest hotel: Ainos Hotel
bukama - nearest hotel: Mission catholique
waterloo - nearest hotel: Delta Waterloo
vallenar - nearest hotel: Hotel del Marqués


,City,Country,Lat,Lng,Humidity,Hotel Name
11,richards bay,ZA,-28.7830,32.0377,52,SUN1 Richards Bay
17,kalemie,CD,-5.9475,29.1947,53,Hotel du Lac
96,ludington,US,43.9553,-86.4526,49,Stearns Hotel
169,nchelenge,ZM,-9.3451,28.7340,39,No hotel found
178,peace river,CA,56.2501,-117.2860,39,New Ridge Inn
190,jamestown,US,42.0970,-79.2353,40,DoubleTree Jamestown
299,surab,PK,28.4914,66.2585,33,No hotel found
304,nantucket,US,41.2835,-70.0995,40,Faraway Nantucket
331,uchiza,PE,-8.4592,-76.4633,47,No hotel found
375,kapoeta,SS,4.7717,33.5903,46,Kuleu


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display


# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    frame_width = 800,
    frame_height = 600,
    hover_cols = ["City", "Country", "Hotel Name"]
)
# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)